<h1> FEATURE SELECTION

In [15]:
import numpy as np
import pandas as pd 

In [16]:
sc_df = pd.read_csv("suicide.csv")

In [17]:
sc_df.head()

,Country,Sex,80,70,60,50,40,30,20,10
0,Afghanistan,Both sexes,42.0,11.0,5.5,5.6,6.6,9.2,10.2,3.1
1,Afghanistan,Male,70.4,20.9,9.8,9.3,10.5,15.1,16.3,4.8
2,Afghanistan,Female,20.1,2.3,1.4,1.6,2.3,2.7,3.5,1.2
3,Albania,Both sexes,16.3,8.3,6.0,7.8,9.1,6.1,6.5,5.0
4,Albania,Male,23.2,11.9,8.1,11.4,13.5,8.8,6.3,3.1


In [18]:
numcols = ['80','70','60','50','40','30','20','10']
catcols = ['Country','Sex']

In [19]:
sc_df = sc_df[numcols+catcols]

In [20]:
sc_df.head()

,80,70,60,50,40,30,20,10,Country,Sex
0,42.0,11.0,5.5,5.6,6.6,9.2,10.2,3.1,Afghanistan,Both sexes
1,70.4,20.9,9.8,9.3,10.5,15.1,16.3,4.8,Afghanistan,Male
2,20.1,2.3,1.4,1.6,2.3,2.7,3.5,1.2,Afghanistan,Female
3,16.3,8.3,6.0,7.8,9.1,6.1,6.5,5.0,Albania,Both sexes
4,23.2,11.9,8.1,11.4,13.5,8.8,6.3,3.1,Albania,Male


In [21]:
traindf = pd.concat([sc_df[numcols], pd.get_dummies(sc_df[catcols])],axis=1)
features = traindf.columns

traindf = traindf.dropna()

In [22]:
traindf = pd.DataFrame(traindf,columns=features)

In [23]:
y = traindf['80']>=87
x = traindf.copy()
del x['80']

In [24]:
x.shape

(549, 193)

In [25]:
y.tail()

544     True
545    False
546     True
547     True
548     True
Name: 80, dtype: bool

In [26]:
len(x.columns)

193

In [27]:
num_feats=30

In [28]:
feature_name = list(x.columns)
feature_name

['70',
 '60',
 '50',
 '40',
 '30',
 '20',
 '10',
 'Country_Afghanistan',
 'Country_Albania',
 'Country_Algeria',
 'Country_Angola',
 'Country_Antigua and Barbuda',
 'Country_Argentina',
 'Country_Armenia',
 'Country_Australia',
 'Country_Austria',
 'Country_Azerbaijan',
 'Country_Bahamas',
 'Country_Bahrain',
 'Country_Bangladesh',
 'Country_Barbados',
 'Country_Belarus',
 'Country_Belgium',
 'Country_Belize',
 'Country_Benin',
 'Country_Bhutan',
 'Country_Bolivia (Plurinational State of)',
 'Country_Bosnia and Herzegovina',
 'Country_Botswana',
 'Country_Brazil',
 'Country_Brunei Darussalam',
 'Country_Bulgaria',
 'Country_Burkina Faso',
 'Country_Burundi',
 'Country_Cabo Verde',
 'Country_Cambodia',
 'Country_Cameroon',
 'Country_Canada',
 'Country_Central African Republic',
 'Country_Chad',
 'Country_Chile',
 'Country_China',
 'Country_Colombia',
 'Country_Comoros',
 'Country_Congo',
 'Country_Costa Rica',
 'Country_Croatia',
 'Country_Cuba',
 'Country_Cyprus',
 'Country_Czechia',
 

<h1> FILTER BASED PEARSON CORRELATION

In [29]:
def cor_selector(x, y,num_feats):
    cor_list = []
    feature_name = x.columns.tolist()
    # calculate the correlation with y for each feature
    for i in x.columns.tolist():
        cor = np.corrcoef(x[i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = x.iloc[:,np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature
cor_support, cor_feature = cor_selector(x, y,num_feats)
print(str(len(cor_feature)), 'selected features')

30 selected features


In [30]:
cor_feature

['Country_Guinea',
 'Country_Ghana',
 'Sex_ Both sexes',
 'Country_Ethiopia',
 'Country_Togo',
 'Country_Equatorial Guinea',
 "Country_Côte d'Ivoire",
 'Country_Cameroon',
 'Country_Lesotho',
 'Country_Senegal',
 'Country_Sierra Leone',
 'Country_Malawi',
 'Country_Cabo Verde',
 'Country_Burundi',
 'Country_Rwanda',
 'Country_Burkina Faso',
 'Country_Benin',
 '10',
 'Country_Liberia',
 'Country_Uganda',
 'Country_Nigeria',
 'Country_Zimbabwe',
 '20',
 'Sex_ Female',
 '30',
 'Sex_ Male',
 '40',
 '50',
 '60',
 '70']

<h1> FILTER BASED CHI SQUARE FEATURES

In [31]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
x_norm = MinMaxScaler().fit_transform(x)
chi_selector = SelectKBest(chi2, k=num_feats)
chi_selector.fit(x_norm, y)
chi_support = chi_selector.get_support()
chi_feature = x.loc[:,chi_support].columns.tolist()
print(str(len(chi_feature)), 'selected features')

30 selected features


In [32]:
chi_feature

['70',
 '60',
 '50',
 '40',
 '30',
 '20',
 '10',
 'Country_Benin',
 'Country_Burkina Faso',
 'Country_Burundi',
 'Country_Cabo Verde',
 'Country_Cameroon',
 "Country_Côte d'Ivoire",
 'Country_Equatorial Guinea',
 'Country_Ethiopia',
 'Country_Lesotho',
 'Country_Liberia',
 'Country_Malawi',
 'Country_Nigeria',
 'Country_Rwanda',
 'Country_Senegal',
 'Country_Sierra Leone',
 'Country_Togo',
 'Country_Uganda',
 'Country_Uruguay',
 'Country_Zambia',
 'Country_Zimbabwe',
 'Sex_ Both sexes',
 'Sex_ Female',
 'Sex_ Male']

<h1>Recursive Feature Elimination

In [33]:

from sklearn.feature_selection import RFE

from sklearn.linear_model import LogisticRegression

rfe_selector = RFE(estimator=LogisticRegression(), n_features_to_select=num_feats, step=10, verbose=5)

rfe_selector.fit(x_norm, y)

rfe_support = rfe_selector.get_support()

rfe_feature = x.loc[:,rfe_support].columns.tolist()

print(str(len(rfe_feature)), 'selected features')

Fitting estimator with 193 features.


C:\Users\USER\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Fitting estimator with 183 features.
Fitting estimator with 173 features.
Fitting estimator with 163 features.
Fitting estimator with 153 features.
Fitting estimator with 143 features.
Fitting estimator with 133 features.
Fitting estimator with 123 features.
Fitting estimator with 113 features.
Fitting estimator with 103 features.
Fitting estimator with 93 features.
Fitting estimator with 83 features.
Fitting estimator with 73 features.
Fitting estimator with 63 features.
Fitting estimator with 53 features.
Fitting estimator with 43 features.
Fitting estimator with 33 features.
30 selected features


C:\Users\USER\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\USER\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\USER\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\USER\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\USER\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this w

<h1>Lasso SelectFromModel

In [34]:
from sklearn.feature_selection import SelectFromModel

from sklearn.linear_model import LogisticRegression



embeded_lr_selector = SelectFromModel(LogisticRegression(penalty="l1"), max_features=num_feats)

embeded_lr_selector.fit(x_norm, y)



embeded_lr_support = embeded_lr_selector.get_support()

embeded_lr_feature = x.loc[:,embeded_lr_support].columns.tolist()

print(str(len(embeded_lr_feature)), 'selected features')

5 selected features


C:\Users\USER\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


<h1>Tree-based: SelectFromModel

In [35]:
from sklearn.feature_selection import SelectFromModel

from sklearn.ensemble import RandomForestClassifier



embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100), max_features=num_feats)

embeded_rf_selector.fit(x, y)



embeded_rf_support = embeded_rf_selector.get_support()

embeded_rf_feature = x.loc[:,embeded_rf_support].columns.tolist()

print(str(len(embeded_rf_feature)), 'selected features')

16 selected features


In [37]:
!pip install lightgbm

In [38]:
from sklearn.feature_selection import SelectFromModel

from lightgbm import LGBMClassifier



lgbc=LGBMClassifier(n_estimators=500, learning_rate=0.05, num_leaves=32, colsample_bytree=0.2,

            reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)



embeded_lgb_selector = SelectFromModel(lgbc, max_features=num_feats)

embeded_lgb_selector.fit(x, y)



embeded_lgb_support = embeded_lgb_selector.get_support()

embeded_lgb_feature = x.loc[:,embeded_lgb_support].columns.tolist()

print(str(len(embeded_lgb_feature)), 'selected features')

30 selected features


In [39]:
# put all selection together

feature_selection_df = pd.DataFrame({'Feature':feature_name, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support, 'Logistics':embeded_lr_support,

                                    'Random Forest':embeded_rf_support, 'LightGBM':embeded_lgb_support})

# count the selected times for each feature

feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)

# display the top 100

feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)

feature_selection_df.index = range(1, len(feature_selection_df)+1)

feature_selection_df.head(num_feats)

,Feature,Pearson,Chi-2,RFE,Logistics,Random Forest,LightGBM,Total
1,70,True,True,True,True,True,True,6
2,20,True,True,True,True,True,True,6
3,Sex_ Female,True,True,True,True,True,False,5
4,Sex_ Both sexes,True,True,True,True,True,False,5
5,60,True,True,True,False,True,True,5
6,50,True,True,True,False,True,True,5
7,10,True,True,True,False,True,True,5
8,Country_Zimbabwe,True,True,True,False,True,False,4
9,Country_Uganda,True,True,True,False,True,False,4
10,Country_Rwanda,True,True,True,False,True,False,4
